In [4]:
from sklearn.neighbors import NearestNeighbors

In [5]:
def fit(X, n_neighbors=3):
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm='ball_tree')
    nbrs.fit(X)
    return nbrs

In [6]:
def score(nbrs, X):
    distances, indices = nbrs.kneighbors(X)
    return distances[:,-1]